<a href="https://colab.research.google.com/github/adi0215/HPE_CTY/blob/main/Practice/Callback_Implementation_NOT_custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mxnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 12.1 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1


In [2]:
import mxnet as mx
from mxnet import gluon, autograd
from mxnet.gluon import nn, loss
import logging

# Define the callback function to log validation metrics

class LogValidationMetricsCallback(object):
    def __call__(self, epoch, param_dict):
        if 'eval_metric' not in param_dict:
            return
        metric = param_dict['eval_metric']
        name_value = metric.get_name_value()
        for name, value in name_value:
            print(f"Epoch[{epoch}] Validation-{name}={value}")
            print(f"Hello  World {epoch}")
# Create the callback instance
log_callback = LogValidationMetricsCallback()
# Define the model architecture
net = nn.Sequential()
with net.name_scope():
    net.add(nn.Dense(128, activation="relu"))
    net.add(nn.Dense(64, activation="relu"))
    net.add(nn.Dense(10))

# Define the model parameters and trainer
ctx = mx.cpu()
net.initialize(ctx=ctx)
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': 0.01})
softmax_cross_entropy = loss.SoftmaxCrossEntropyLoss()

# Load the MNIST dataset
batch_size = 64
train_data = gluon.data.DataLoader(gluon.data.vision.MNIST(train=True, transform=lambda data, label: (data.astype('float32')/255, label)), batch_size, shuffle=True)#splitting data for training
val_data = gluon.data.DataLoader(gluon.data.vision.MNIST(train=False, transform=lambda data, label: (data.astype('float32')/255, label)), batch_size, shuffle=False)#splitting data for validation

# Train the model with the callback
num_epochs = 3
for epoch in range(num_epochs):
    # Train for one epoch
    train_loss = 0
    for data, label in train_data:
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(batch_size)
        train_loss += mx.nd.mean(loss).asscalar()

    # Run validation
    metric = mx.metric.Accuracy()
    val_loss = 0
    for data, label in val_data:
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        output = net(data)
        metric.update(label, output)
        val_loss += softmax_cross_entropy(output, label).mean().asscalar()

    # Log validation metrics using the callback
    log_callback(epoch, {'eval_metric': metric})

    # Print training progress
    print(f"Epoch [{epoch+1}], training loss: {train_loss/len(train_data)}, validation loss: {val_loss/len(val_data)}, validation accuracy: {metric.get()[1]}")


Epoch[0] Validation-accuracy=0.946
Hello  World 0
Epoch [1], training loss: 0.23985991810263793, validation loss: 0.19138883753675243, validation accuracy: 0.946
Epoch[1] Validation-accuracy=0.9647
Hello  World 1
Epoch [2], training loss: 0.14028856268441522, validation loss: 0.13244286478348047, validation accuracy: 0.9647
Epoch[2] Validation-accuracy=0.9672
Hello  World 2
Epoch [3], training loss: 0.11931656475570093, validation loss: 0.11937823896904351, validation accuracy: 0.9672


In [3]:
import mxnet as mx
from mxnet import gluon, autograd
from mxnet.gluon import nn, loss
import logging
import math
# Define the callback function to ProgressBar

class ProgressBar(object):
    def __init__(self, total, length=80):
        self.bar_len = length
        self.total = total

    def __call__(self,epoch,logs=None):
     #   """Callback to Show progress bar."""
        count = logs['batch']
        filled_len = int(round(self.bar_len * count / float(self.total)))
        percents = math.ceil(100.0 * count / float(self.total))
        prog_bar = '=' * filled_len + '-' * (self.bar_len - filled_len)
        print(f'Epoch [{epoch+1}] |{prog_bar}| {percents}%')

# Define the model architecture
net = nn.Sequential()
with net.name_scope():
    net.add(nn.Dense(128, activation="relu"))
    net.add(nn.Dense(64, activation="relu"))
    net.add(nn.Dense(10))

# Define the model parameters and trainer
ctx = mx.cpu()
net.initialize(ctx=ctx)
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': 0.01})
softmax_cross_entropy = loss.SoftmaxCrossEntropyLoss()

# Load the MNIST dataset
batch_size = 64
train_data = gluon.data.DataLoader(gluon.data.vision.MNIST(train=True, transform=lambda data, label: (data.astype('float32')/255, label)), batch_size, shuffle=True)
val_data = gluon.data.DataLoader(gluon.data.vision.MNIST(train=False, transform=lambda data, label: (data.astype('float32')/255, label)), batch_size, shuffle=False)

# Create the callback instance
log_callback = ProgressBar(total=len(train_data))

# Train the model with the callback
num_epochs = 3
for epoch in range(num_epochs):
    # Train for one epoch
    train_loss = 0
    for i,( data,label) in enumerate(train_data):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(batch_size)
        train_loss += mx.nd.mean(loss).asscalar()
        #Update Progress Bar
        log_callback(epoch, {'batch': i})
    # Run validation
    metric = mx.metric.Accuracy()
    val_loss = 0
    for data, label in val_data:
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        output = net(data)
        metric.update(label, output)
        val_loss += softmax_cross_entropy(output, label).mean().asscalar()



    # Print training progress
    print(f"Epoch [{epoch+1}], training loss: {train_loss/len(train_data)}, validation loss: {val_loss/len(val_data)}, validation accuracy: {metric.get()[1]}")


Epoch [1] |--------------------------------------------------------------------------------| 0%
Epoch [1] |--------------------------------------------------------------------------------| 1%
Epoch [1] |--------------------------------------------------------------------------------| 1%
Epoch [1] |--------------------------------------------------------------------------------| 1%
Epoch [1] |--------------------------------------------------------------------------------| 1%
Epoch [1] |--------------------------------------------------------------------------------| 1%
Epoch [1] |=-------------------------------------------------------------------------------| 1%
Epoch [1] |=-------------------------------------------------------------------------------| 1%
Epoch [1] |=-------------------------------------------------------------------------------| 1%
Epoch [1] |=-------------------------------------------------------------------------------| 1%
Epoch [1] |=----------------------------